# Schneider Electrick Hackaton - Data Science
## Team 4 - Gerard Jover Pujol - Pol Gràcia Espelt

### Modules import

In [162]:
import os
import sklearn as sk
import pandas as pd
import numpy as np 
import requests as r
import pandas_profiling

### Data load

In [163]:
train_data_dir_csv = ['Data/CSV/' + x for x in os.listdir('Data/CSV') if 'train' in x]
test_data_dir_csv = ['Data/CSV/' + x for x in os.listdir('Data/CSV') if 'test' in x][0]
train_data_dir_pdf = ['Data/pdf/' + x for  x in os.listdir('Data/pdf')]
train_data_api = 'Data/json/links.txt'

#### CSV

In [218]:
train_csv = pd.concat([pd.read_csv(train_data_dir_csv[0]), pd.read_csv(train_data_dir_csv[1], sep = ';')])

In [219]:
train_csv.count()

countryName                     37127
eprtrSectorName                 37127
EPRTRAnnexIMainActivityLabel    37127
FacilityInspireID               37127
facilityName                    37127
City                            37127
targetRelease                   37127
pollutant                       37127
reportingYear                   37127
MONTH                           37127
DAY                             37127
CONTINENT                       37127
max_wind_speed                  37127
avg_wind_speed                  37127
min_wind_speed                  37127
max_temp                        37127
avg_temp                        37127
min_temp                        37127
DAY WITH FOGS                   37127
REPORTER NAME                   37127
CITY ID                         37127
dtype: int64

In [214]:
train_csv[train_csv['facilityName'] == 'Fife Ethylene Plant']

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
3874,United Kingdom,Energy sector,Thermal power stations and other combustion in...,UK.SEPA/200000061.Facility,Fife Ethylene Plant,Cowdenbeath,AIR,Nitrogen oxides (NOX),2019,12,...,EUROPE,15.141264,18.269775,20.029105,8.923795,11.725372,15.124218,9,Bruce Farrell,3c563ab0d76fc84128574b5da82f769a


### api

In [217]:
with open(train_data_api) as file:
    links = [link.strip() for link in file]

int_cols = ['MONTH','reportingYear', 'DAY', 'max_wind_speed', 'avg_wind_speed', 'min_wind_speed', 'max_temp', 'min_temp', 'avg_temp', 'DAY WITH FOGS']
json_df = pd.DataFrame()
for link in links:
    data_json = r.get(link).json()
    tmp_df = pd.json_normalize(data_json)
    #set index
    tmp_df.set_index(tmp_df[''], inplace = True)
    tmp_df.drop('', axis = 1, inplace=True)
    #values to int
    tmp_df[int_cols] = tmp_df[int_cols].apply(pd.to_numeric)
    #print(tmp_df.columns)
    #concat
    json_df = pd.concat([json_df, tmp_df])
    #train_csv = pd.concat([train_csv, tmp_df])

In [224]:
c = 0
l = list(json_df['facilityName'])
for x in train_csv.facilityName:
    if x not in l:
        c+=1


In [228]:
len(set(train_csv['facilityName'].unique()) - set(json_df['facilityName'].unique()))

1239

In [229]:
len(train_csv['facilityName'].unique())

6704

In [226]:
len(json_df['facilityName'].unique())

6691

In [242]:
pd.DataFrame(df_bo[['eprtrSectorName', 'EPRTRSectorCode']].value_counts()).reset_index()

,eprtrSectorName,EPRTRSectorCode,0
0,Energy sector,1,10648
1,Waste and wastewater management,5,6959
2,Mineral industry,3,4554
3,Chemical industry,4,1810
4,Paper and wood production and processing,6,1610
5,Production and processing of metals,2,1376
6,Intensive livestock production and aquaculture,7,916
7,Animal and vegetable products from the food an...,8,525
8,Other activities,9,103


In [230]:
df_bo = pd.concat([train_csv, json_df])

In [231]:
df_bo[['eprtrSectorName', 'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode']].value_counts().to_csv('hla.csv')

In [198]:
train_csv['EPRTRAnnexIMainActivityCode'].unique()

array([nan, '4(a)', '3(c)', '1(c)', '7(a)', '5(a)', '3(c)(i)', '5(b)',
       '5(d)', '7(a)(iii)', '4(b)', '8(a)', '2(c)(i)', '4(a)(viii)',
       '2(e)(ii)', '7(a)(ii)', '1(a)', '3(e)', '5(e)', '6(b)', '2(a)',
       '1(b)', '8(b)', '3(a)', '2(b)', '5(c)', '4(e)', '5(f)', '4(a)(ii)',
       '4(a)(x)', '6(a)', '8(b)(ii)', '8(c)', '2(e)', '1(f)', '3(b)',
       '3(g)', '4(b)(ii)', '5(g)', '3(c)(iii)', '4(a)(i)', '4(b)(v)',
       '2(e)(i)', '2(f)', '1(d)', '3(c)(ii)', '6(c)', '2(c)(ii)',
       '7(a)(i)', '4(b)(iv)', '2(d)', '4(b)(iii)', '4(c)', '4(b)(i)',
       '2(c)', '4(a)(iv)', '9(c)', '4(a)(iii)', '2(c)(iii)', '4(a)(xi)',
       '3(f)', '9(a)', '4(a)(vi)', '4(f)', '9(d)', '4(d)', '1(e)', '9(e)',
       '4(a)(v)', '8(b)(i)', '4(a)(ix)'], dtype=object)

In [199]:
train_csv['EPRTRSectorCode'].unique()

array([nan, '4', '3', '1', '7', '5', '8', '2', '6', '9'], dtype=object)

In [168]:
set(train_csv.columns) - set(pd.read_csv(test_data_dir_csv).columns)

{'pollutant'}

In [169]:
pd.read_csv(test_data_dir_csv).columns

Index(['test_index', 'countryName', 'EPRTRSectorCode', 'eprtrSectorName',
       'EPRTRAnnexIMainActivityCode', 'EPRTRAnnexIMainActivityLabel',
       'FacilityInspireID', 'facilityName', 'City', 'targetRelease',
       'reportingYear', 'MONTH', 'DAY', 'CONTINENT', 'max_wind_speed',
       'avg_wind_speed', 'min_wind_speed', 'max_temp', 'avg_temp', 'min_temp',
       'DAY WITH FOGS', 'REPORTER NAME', 'CITY ID'],
      dtype='object')

In [170]:
train_csv.dtypes

countryName                      object
eprtrSectorName                  object
EPRTRAnnexIMainActivityLabel     object
FacilityInspireID                object
facilityName                     object
City                             object
targetRelease                    object
pollutant                        object
reportingYear                     int64
MONTH                             int64
DAY                               int64
CONTINENT                        object
max_wind_speed                  float64
avg_wind_speed                  float64
min_wind_speed                  float64
max_temp                        float64
avg_temp                        float64
min_temp                        float64
DAY WITH FOGS                     int64
REPORTER NAME                    object
CITY ID                          object
EPRTRAnnexIMainActivityCode      object
EPRTRSectorCode                  object
dtype: object

In [174]:
pr = train_csv.profile_report()

In [175]:
pr.to_file('report_train.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 42.79it/s]


### Models

### Training

### Testing